# Multiple tools

This example shows how to add multiple interactive tools to the toolbar.
We add both the `InspectTool` to inspect the `zz` dimension of the data,
and the `LineCutTool` to draw cuts onto the image.

Each tool requires a one-dimensional figure where it can display its results.

In [ ]:
%matplotlib widget
import plopp as pp
import scipp as sc
from plopp import widgets

In [ ]:
# Create data
da = pp.data.data_array(ndim=3, binedges=True)

# Input node for raw data
in_node = pp.input_node(da)
# Apply sum along 'zz' dimension
sum_node = pp.node(sc.sum, dim='zz')(in_node)
# Main figure for image plot
fig2d = pp.figure2d(sum_node)

# Add inspection tool
inspect_fig = pp.figure1d(title='Inspect')
fig2d.add_tool(widgets.InspectTool, root_node=in_node, fig1d=inspect_fig)

# Add line-cut tool
cut_fig = pp.figure1d(title='Line cuts', ls='solid', marker=None)
fig2d.add_tool(widgets.LineCutTool, fig1d=cut_fig)

In [ ]:
import numpy as np
from dataclasses import dataclass

np.random.seed(78)
N = 3

@dataclass
class Event:
    xdata: float
    ydata: float

tool = fig2d.toolbar['InspectTool']
for i in range(N):
    data = 30.0 * np.random.random(2)
    tool.points._make_new_line(data[0], data[1])
    tool.points._finalize_line(event=Event(xdata=data[0], ydata=data[1]))

N = 2
tool = fig2d.toolbar['LineCutTool']
for i in range(N):
    x = 30.0 * np.random.random(2)
    y = 30.0 * np.random.random(2)
    tool.lines._make_new_line(x[0], y[0])
    l = tool.lines.lines[-1]
    new_data = l.get_data()
    new_data[0][-1] = x[1]
    new_data[1][-1] = y[1]
    l.set_data(new_data)
    tool.lines._finalize_line(event=Event(xdata=x[1], ydata=y[1]))
    
def update(*figs):
    from ipywidgets import HBox
    for f in figs:
        # f1 = pl.children[0]
        f.children = [f.top_bar, HBox([f.left_bar, f.canvas.to_image(), f.right_bar]), f.bottom_bar]
    # f2 = pl.children[1]
    # f2.children = [f2.top_bar, HBox([f2.left_bar, f2.canvas.to_image(), f2.right_bar]), f2.bottom_bar]

update(fig2d, inspect_fig, cut_fig)

In [ ]:
# Display all three figures together
widgets.Box([fig2d, inspect_fig, cut_fig])

In [ ]:
# This cell is used to generate the thumbnail for the docs gallery.
# It is hidden from the online documentation.
fig2d.canvas.savefig('../../_static/gallery/multiple-tools-thumbnail.png')